# Dummy Net

**Interesting tasks**
 - inspect tensorboard
 - change learning rates
 - change initializer

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from utils.data import init_dir
import tensorflow as tf
import numpy as np
import os

## Build Net Graph

In [6]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('dummy_net_inputs'):
        data = tf.placeholder(tf.float32, shape=[None, 3], name='data')
        labels = tf.placeholder(tf.float32, shape=[None, 1], name='labels')
    
    with tf.variable_scope('layer_1'):
        w1 = tf.get_variable('w1', initializer=tf.constant_initializer(0), shape=[3,2])
        prediction = tf.matmul(data, w1)
        tf.summary.histogram('w1_histogram', w1)
    
    with tf.variable_scope('layer_2'):
        w2 = tf.get_variable('w2', initializer=tf.truncated_normal(shape=[2,1], stddev=0.1))
        prediction = tf.matmul(prediction, w2)
        print('Debug [prediction]:', prediction.get_shape())
        tf.summary.histogram('w2_histogram', w2)
    
    with tf.name_scope('loss'):
        loss = tf.losses.mean_squared_error(labels, prediction)
        tf.summary.scalar('loss_summary', loss)

    with tf.name_scope('training'):
        train_step = tf.train.GradientDescentOptimizer(learning_rate=0.005).minimize(loss)

    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

Debug [prediction]: (?, 1)


In [9]:
graph.get_all_collection_keys()

['trainable_variables', 'train_op', 'losses', 'summaries', 'variables']

## Init Model Logging

In [11]:
from utils.data import init_model_logging
base_dir = '/tensorboard_summaries/dummy_net/'

logging_meta = init_model_logging(base_dir, 'experiment1', graph=graph, remove_existing=True)

In [12]:
logging_meta

{'model_path': '/tensorboard_summaries/dummy_net/experiment1/valid/model.ckpt',
 'saver': <tensorflow.python.training.saver.Saver at 0x7f0200e064a8>,
 'train_writer': <tensorflow.python.summary.writer.writer.FileWriter at 0x7f0200e06588>,
 'train_writer_dir': '/tensorboard_summaries/dummy_net/experiment1/train',
 'valid_writer': <tensorflow.python.summary.writer.writer.FileWriter at 0x7f0200e06828>,
 'valid_writer_dir': '/tensorboard_summaries/dummy_net/experiment1/valid'}

## Run net

In [13]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True

_data = np.array([[1,2,3]])
_labels = np.array([[5]])

with tf.Session(graph=graph, config=config) as session:
    session.run(initialize_vars)
    
    for iteration in range(100):
        # Prepare data
        feed_dict = {data: _data, labels: _labels}
        fetches = [train_step, prediction, loss, merge_summaries]

        # Train
        _, _prediction, _loss, _summary = session.run(fetches, feed_dict)
        
        # Log
        logging_meta['train_writer'].add_summary(_summary, iteration)
        print("Iteration {}: loss {}, pred {}".format(iteration, _loss, _prediction[0][0]))

    logging_meta['saver'].save(session, logging_meta['model_path'])

Iteration 0: loss 25.0, pred 0.0
Iteration 1: loss 24.779279708862305, pred 0.022121001034975052
Iteration 2: loss 24.545263290405273, pred 0.04568234458565712
Iteration 3: loss 24.267642974853516, pred 0.07378026843070984
Iteration 4: loss 23.91391372680664, pred 0.1098146140575409
Iteration 5: loss 23.44634437561035, pred 0.15785764157772064
Iteration 6: loss 22.819705963134766, pred 0.2230023741722107
Iteration 7: loss 21.98041343688965, pred 0.3116726875305176
Iteration 8: loss 20.868255615234375, pred 0.431821346282959
Iteration 9: loss 19.42290687561035, pred 0.592857301235199
Iteration 10: loss 17.59773826599121, pred 0.805034339427948
Iteration 11: loss 15.382562637329102, pred 1.0779390335083008
Iteration 12: loss 12.832423210144043, pred 1.4177627563476562
Iteration 13: loss 10.090475082397461, pred 1.8234491348266602
Iteration 14: loss 7.383357524871826, pred 2.282766580581665
Iteration 15: loss 4.97019100189209, pred 2.7706074714660645
Iteration 16: loss 3.0549468994140625,